# Script auto distance for data

- Made a small script for calculating distance Ploop_bend6. 
    - Can easily be adopted for other distances.
    - Anchor point interpreters can be introduced.
    - Not reliable yet, due to static choice of atom, also static choice of residue number
- Tested script on a few proteins
    - did measurements for all proteins in the PKACa folder
    - compared results with reference excel sheet with good results
    - probably still room for improvement

## Code

### Imports

In [1]:
import numpy as np
import pandas as pd
import re
import os
from sympy import Plane, Point3D


### Dataclass

In [81]:

class molData:
    def __init__(self,path,data_section = 0):
        my_data = pd.read_table(path,skiprows=8,header=None,delim_whitespace=True,index_col=0)
        atoms = my_data.loc[:"@<TRIPOS>BOND"].iloc[:-1]
        atoms.columns = ["atomID","x","y","z","compound2",
                         "residueNR","residue","charge","mainChain"]
        resPattern = r"[A-Z]{3}[0-9]*"
        hohPattern = r"HOH[0-9]*"
        residueMask = atoms.loc[:,"residue"].str.match(resPattern,as_indexer=True).values
        hohMask = atoms.loc[:,"residue"].str.match(hohPattern,as_indexer=True).values
        self.other = atoms.loc[np.logical_not(residueMask)]
        self.water = atoms.loc[hohMask]
        self.chain = atoms.loc[np.logical_and(residueMask,np.logical_not(hohMask))]
        resDigest = self.chain.loc[:,"residue"].str.extract('(?P<residueType>[A-Z]{3})(?P<residueID>[0-9]*)')
        self.chain = self.chain.join(resDigest)
        self.chainIDGroups = self.chain.groupby(["residueID","atomID"])
        self.chainNRGroups = self.chain.groupby(["residueNR","atomID"])
        self.bonds = my_data.loc["@<TRIPOS>BOND":"@<TRIPOS>SUBSTRUCTURE"].iloc[1:-1]
        self.subst = my_data.loc["@<TRIPOS>SUBSTRUCTURE":].iloc[1:-2]

    def anchordistance(self,res1,res2):
        coord1 = self.get_atom_coords(res1)
        coord2 = self.get_atom_coords(res2)
        return self.distance(coord1,coord2)
        
    def distance(self,coord1,coord2):
        return np.linalg.norm(coord1 - coord2)
    
    def old_get_atom_coords(self, residueID, anchor_atom=2):
        """
            extracts coordinates for target atom according to residue type (iloc may need to be variable)
        """
        residue = self.chainIDGroups.get_group((residueID))
        anchor = residue.iloc[anchor_atom]
        coords = anchor.loc["x":"z"]
        return np.array(coords).astype(np.float32)
    
    def get_atom_coords1(self, residueID, anchor_atom="CA"): # combine with 2 later
        anchor = self.chainIDGroups.get_group((residueID,anchor_atom))
        coords = anchor.loc[:,"x":"z"]
        return np.array(coords).astype(np.float32)

    def get_atom_coords2(self, residueNR, anchor_atom="CA"):
        anchor = self.chainNRGroups.get_group((residueNR,anchor_atom))
        coords = anchor.loc[:,"x":"z"]
        result = np.array(coords.values).astype(np.float32)[0]
        return result
        
    def chi_angle(self,coords):
        plane1 = Plane(Point3D(coords[0]), Point3D(coords[1]), Point3D(coords[2]))
        plane2 = Plane(Point3D(coords[1]), Point3D(coords[2]), Point3D(coords[3]))
        return plane1.angle_between(plane2)
        
    def calculate_chi1(self,residue):
        Amcrd = self.get_atom_coords2(residue, "N")
        CAcrd = self.get_atom_coords2(residue, "CA")
        CBcrd = self.get_atom_coords2(residue, "CB")
        CGcrd = self.get_atom_coords2(residue, "CG")
        coords = (Amcrd, CAcrd, CBcrd, CGcrd)
        return self.chi_angle(coords).evalf()
    
    def calculate_chi2(self,residue):
        CAcrd = self.get_atom_coords2(residue, "CA")
        CBcrd = self.get_atom_coords2(residue, "CB")
        CGcrd = self.get_atom_coords2(residue, "CG")
        SDcrd = self.get_atom_coords2(residue, "SD")
        coords = (CAcrd, CBcrd, CGcrd, SDcrd)
        return self.chi_angle(coords).evalf()
    
    def chi_angles(self,residue):
        chi1 = self.calculate_chi1(str(residue))
        chi2 = self.calculate_chi2(str(residue))
        return [chi1, chi2]

    

### Calculate distances for a bunch of proteins

In [22]:

DATASECTIONS = {
    0: "/complex.mol2",
    1: "/pocket.mol2"
}

def calculate_protein_data(kinase_group_path, data_section, measurement, res_ids):
    kinases = pd.Series([name for name in os.listdir(path1) if os.path.isdir(path1+name)])
    kinase_labels = kinases.str.replace("chain","").str.lower().str.replace("_alt[ab]","")
    results = pd.DataFrame(index = kinase_labels)
    results["measured"] = pd.Series()

    for kinase, kinase_label in zip(kinases,kinase_labels):
        myurl = path1 + kinase + DATASECTIONS[data_section]
        mydata = molData(myurl,data_section)
        if measurement == "distance":
            residueID1 = res_ids[0]
            residueID2 = res_ids[1]
            results.loc[kinase_label,"measured"] = mydata.anchordistance(residueID1,residueID2)
        elif measurement == "chi_angles":
            residueID = res_ids
            results.loc[kinase_label,"measured"] = mydata.chiangles(residueID)
    return results

path1 = "/Users/matthias/Documents/hibit.in/KLIFS_LigandBound/HUMAN/PKACa/"
data1 = 0
meas1 = "distance"
ress1 = ["54","57"]

results = calculate_protein_data(path1, data1, meas1, ress1)

results = results.drop_duplicates()


/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:15: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)


### Compare with excel

In [111]:

#separate because takes time

reference = pd.read_excel("/Users/matthias/Documents/hibit.in/KinaseDomainMeasures-Novartis.xlsx",
                          header=0,
                          sheetname=0,
                          parse_cols="B,C,AJ")



In [147]:

ref_mod = reference.set_index("pdbchain",drop=False).loc[results.index]
ref_mod.loc[:,"pdbchain_unique"] = ref_mod.loc[:,"pdbchain_unique"].str.replace(r'[\S]{4}_[abe]_','')
ref_mod = ref_mod.drop_duplicates()
ref_mod = ref_mod.pivot(index="pdbchain",columns="pdbchain_unique",values="Ploop_bend6").iloc[1:,1:]
end_results = pd.concat([results,ref_mod],axis=1)

end_results["diff1"] = end_results["measured"].sub(end_results["pdb"],axis=0)
end_results["diff2"] = end_results["measured"].sub(end_results["pdbredo"],axis=0)
end_results

,measured,pdb,pdbredo,diff1,diff2
2gu8_a,10.616140,10.61610,10.61070,0.000040,0.005440
3agl_a,10.771265,10.77120,10.76240,0.000065,0.008865
3agl_b,10.741295,10.74130,10.75990,-0.000005,-0.018605
3ama_a,9.539944,9.54004,9.52416,-0.000096,0.015784
3amb_a,9.597634,9.59758,9.58747,0.000054,0.010164
3l9l_a,9.096610,9.09654,9.02010,0.000070,0.076510
3l9l_b,8.899876,8.89988,9.03277,-0.000004,-0.132894
3l9m_a,8.446536,8.44650,8.93785,0.000036,-0.491314
3l9m_b,8.862170,8.86213,8.96780,0.000040,-0.105630
3l9n_a,9.361628,9.36162,9.33039,0.000008,0.031238


## Calculate metheonin gatekeeper chi angles

In [82]:
myurl = "/Users/matthias/Documents/hibit.in/KLIFS_LigandBound/HUMAN/PKACa/2gu8_chainA/pocket.mol2"


mydata = molData(myurl)

print mydata.chi_angles(45)


[1.01297476476367, 2.83046046933386]


/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:15: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
